In [32]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.normalization import BatchNormalization
import numpy as np
import random
from keras import backend as K
smooth = 1
import cv2
import os
import pickle
from keras.engine.topology import Input
from keras.layers import concatenate,Conv2DTranspose
if not os.path.exists('pickled'):
    os.makedirs('pickled')
print("Start")

Start


In [33]:
print("loading images...")
data = []
labels = []
if not os.path.exists('pickled/seg_data.pkl'):
    directory = os.fsencode("melanoma")
    for file in os.listdir(directory):
        filename = os.fsdecode(file) 
        
        path = str('melanoma\\' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)

    directory = os.fsencode("others")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('others\\' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt\\' + filename.split('.')[0] + '_segmentation.png')
        
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        labels.append(image)
        
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickled/seg_data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickled/seg_data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']
                
print("Done.")

loading images...
Done.


In [34]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels, dtype="float") / 255.0

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)
print("Done")

Done


In [35]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

img_input = Input((img_rows, img_cols, 3))

x = img_input
x = Conv2D(64, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(256, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(512, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

x = Conv2D(512, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(256, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(128, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(64, (3, 3), padding="same")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

x = Conv2D(3, (1, 1), padding="valid")(x)
x = Activation("sigmoid")(x)
model = Model(img_input, x)

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Sørensen–Dice_coefficient score :', score[1])

1600 train samples
400 test samples
Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 1162s 726ms/step - loss: -0.4317 - dice_coef: 0.4317 - val_loss: -0.4929 - val_dice_coef: 0.4929
Epoch 2/10
1600/1600 [==============================] - 1186s 741ms/step - loss: -0.5226 - dice_coef: 0.5226 - val_loss: -0.5446 - val_dice_coef: 0.5446
Epoch 3/10
1600/1600 [==============================] - 1205s 753ms/step - loss: -0.5611 - dice_coef: 0.5611 - val_loss: -0.5825 - val_dice_coef: 0.5825
Epoch 4/10
1600/1600 [==============================] - 1151s 719ms/step - loss: -0.5860 - dice_coef: 0.5860 - val_loss: -0.6112 - val_dice_coef: 0.6112
Epoch 5/10
1600/1600 [==============================] - 1269s 793ms/step - loss: -0.6001 - dice_coef: 0.6001 - val_loss: -0.6161 - val_dice_coef: 0.6161
Epoch 6/10
1600/1600 [==============================] - 1104s 690ms/step - loss: -0.6125 - dice_coef: 0.6125 - val_loss: -0.6331 - val_dice_coef: 0.6331